In [1]:
!pip install flask tensorflow opencv-python


In [ ]:
# ✅ Flask app for live camera + classifier
from flask import Flask, render_template, Response
import cv2
import tensorflow as tf
import numpy as np

# Load your trained classifier
model_path = r"C:\Users\sagni\Downloads\Trash-Detector\waste_classifier.h5"
model = tf.keras.models.load_model(model_path)
class_labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

# Preprocess frame
def preprocess_frame(frame):
    img = cv2.resize(frame, (224, 224))
    img = img.astype('float32') / 255.0
    return np.expand_dims(img, axis=0)

# Predict class
def get_prediction(frame):
    input_img = preprocess_frame(frame)
    preds = model.predict(input_img)
    class_idx = np.argmax(preds)
    confidence = np.max(preds)
    return f"{class_labels[class_idx]} ({confidence*100:.2f}%)"

# Flask app
app = Flask(__name__)

# Camera feed generator
def gen_frames():
    cap = cv2.VideoCapture(0)
    while True:
        success, frame = cap.read()
        if not success:
            break
        else:
            # Predict and overlay
            prediction = get_prediction(frame)
            cv2.putText(frame, prediction, (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Encode frame for HTML
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

# Routes
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video')
def video():
    return Response(gen_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

# Run app
if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.5.50.65:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sagni\AppData\Local\